In [102]:
import jiwer
import pprint
import os
import re
import json
import sys
import numpy as np
from tools.utils import remove_emoji, get_song_list, compute_wer, convert_digits_to_words
from tools.utils import LANGUAGES, transformation
import glob

def add_song(song_name, _dict, all, pred_dir, gt_dir):
    _dict[song_name] = {}

    # transcribed = os.path.join(pred_dir, f'{song_name}.{"json" if content_type == "json" else "gptrlt.txt"}')
    # with open(transcribed, 'r') as _file:
    #     if content_type == 'json':
    #         t_text = json.load(_file)['output']
    #     else:
    #         t_text = _file.read()
            # t_text = json.loads(t_text)['output']

    ## compare whisper result with ground truth
    transcribed = os.path.join(pred_dir, song_name + '.mp3.json')
    print('transcribed', transcribed)
    with open(transcribed, 'r') as _file:
        t_segs = json.load(_file)['segments']
        t_text = []
        for seg in t_segs:
            if seg['no_speech_prob'] < 0.9:
                t_text.append(seg['text'])
    t_text = ' '.join(t_text)


    official = os.path.join(gt_dir, f'{song_name}.origin.lrc')
    print(official)
    with open(official, 'r') as _file:
        o_text = _file.read()

    lines = o_text.splitlines()
    cleaned_lines = [re.sub(r"\[.*?\]", "", line) for line in lines] # regex to remove enclosing [..] tags from ground truth
    cleaned_lines = [line for line in cleaned_lines if (':' not in line) and (line != '')]
    o_text = ' '.join(cleaned_lines) # currently join by space, should try "\n" as well
    
    # post processing of the text
    t_text = t_text.lower()
    t_text = t_text.replace('\n', ' ')
    t_text = remove_emoji(t_text)
    t_text = t_text.replace(' am', 'm')
    t_text = t_text.replace('\'d', ' would')
    t_text = transformation(t_text)[0] # Get the first item in the list
    t_text = convert_digits_to_words(t_text)
    
    o_text = o_text.lower()
    o_text = o_text.replace('\n', ' ')
    o_text = remove_emoji(o_text)
    o_text = o_text.replace(' am', 'm')
    o_text = o_text.replace('\'d', ' would')
    o_text = transformation(o_text)[0] # Get the first item in the list
    o_text = convert_digits_to_words(o_text)

    # post processing of for the ground truth
    for i, word in enumerate(o_text):
        if word[-2:] == 'in' and len(word) > 2:
            o_text[i] = word[:-2] + 'ing'

    # print(len(o_text), len(t_text))
    o_text = ' '.join(o_text).strip()
    t_text = ' '.join(t_text).strip()
    errors = jiwer.compute_measures(truth=o_text, hypothesis=t_text)
    cer_errors = jiwer.cer(reference=o_text, hypothesis=t_text)
    _dict[song_name]['mer'] = errors['mer']
    _dict[song_name]['wer'] = errors['wer']
    _dict[song_name]['wil'] = errors['wil']
    _dict[song_name]['wip'] = errors['wip']
    all['mer'] += errors['mer']
    all['wer'] += errors['wer']
    all['wil'] += errors['wil']
    all['wip'] += errors['wip']
    all['cer'] += cer_errors
    
    print()
    print(song_name, "WER: ", errors['wer'], "CER: ", cer_errors)
    print("Prediction: ", len(t_text))
    print(t_text)
    print("Ground truth: ", len(o_text))
    print(o_text)

In [2]:
jiwer.compute_measures(truth="how are you today? I am fine thank you", hypothesis="how is you? I am ok thanks")

{'wer': 0.6666666666666666,
 'mer': 0.6666666666666666,
 'wil': 0.8571428571428572,
 'wip': 0.14285714285714285,
 'hits': 3,
 'substitutions': 4,
 'deletions': 2,
 'insertions': 0,
 'ops': [[AlignmentChunk(type='equal', ref_start_idx=0, ref_end_idx=1, hyp_start_idx=0, hyp_end_idx=1),
   AlignmentChunk(type='substitute', ref_start_idx=1, ref_end_idx=3, hyp_start_idx=1, hyp_end_idx=3),
   AlignmentChunk(type='delete', ref_start_idx=3, ref_end_idx=4, hyp_start_idx=3, hyp_end_idx=3),
   AlignmentChunk(type='equal', ref_start_idx=4, ref_end_idx=6, hyp_start_idx=3, hyp_end_idx=5),
   AlignmentChunk(type='substitute', ref_start_idx=6, ref_end_idx=8, hyp_start_idx=5, hyp_end_idx=7),
   AlignmentChunk(type='delete', ref_start_idx=8, ref_end_idx=9, hyp_start_idx=7, hyp_end_idx=7)]],
 'truth': [['how', 'are', 'you', 'today?', 'I', 'am', 'fine', 'thank', 'you']],
 'hypothesis': [['how', 'is', 'you?', 'I', 'am', 'ok', 'thanks']]}

In [3]:
jiwer.cer(reference="how is you? I am ok thank", hypothesis="how is you? I am ok thanks")

0.04

In [133]:
results = {}
all = {'mer': 0, 'wer': 0, 'wil': 0, 'wip': 0, 'wer2': 0, 'cer': 0}
pred_dir = '../post_processed_transcripts/'
# gt_dir = '../sample/'
gt_dir = '../master_1st_1000/out/master_downloads_1st/'
# add_song("en_sample1", results, all, pred_dir, gt_dir)

In [134]:
preprocessed_song = glob.glob(f'{pred_dir}/*.json')
idx = 0
for song in preprocessed_song:
    song_name = os.path.basename(song).split('.mp3')[0]
    # if song_name in [os.path.basename(x).split('.mp3.json')[0] for x in os.listdir('../../master_1st_100_results_test_only/') if x !=  '.ipynb_checkpoints']:
    add_song(song_name, results, all, pred_dir, gt_dir)
    idx += 1

transcribed ../post_processed_transcripts/Anh Sẽ Mãi Bên Em.mp3.json
../master_1st_1000/out/master_downloads_1st/Anh Sẽ Mãi Bên Em.origin.lrc

Anh Sẽ Mãi Bên Em WER:  0.14058355437665782 CER:  0.08121512709237445
Prediction:  1614
người yêu ơi kể từ ngày mình quen nhau anh đã khát khao cho chúng ta gần nhau vì anh biết tình mình thật đậm sâu chỉ có bên em cho trái tim anh bình yên dù thời gian phải chờ đợi bao lâu hay có bao giọng chia cắt đôi tình ta nguyện một lòng sẽ chẳng hề phối phát anh sẽ nắm tay em chê chê tình yêu và rồi anh ước thật nhiều niềm vui ngang hoa tấm dành tặng cho em người biết không anh hạnh phúc lắm một ngày thiếu vắng nụ cười của em cả thế giới chìm vào bậc sâu chỉ có riêng anh với nỗi buồn cuộc đời sẽ chẳng còn gì đẹp hơn nên hạnh phúc là ngày cùng em được nắm tay nhau say ước mơ người ngàn đế ước cùng anh vì sao luôn sợi sáng chứng minh tình ta anh sẽ bên em cho nơi dù thời gian phải chờ đây bao lâu hay có bao dâu chia cắt đôi tình ta nguyện một lòng sẽ chẳng 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



../master_1st_1000/out/master_downloads_1st/Chờ Đợi Trong Nước Mắt.origin.lrc

Chờ Đợi Trong Nước Mắt WER:  0.4368421052631579 CER:  0.4351676154332701
Prediction:  910
anh vẫn mong chờ ngày em sẽ quay trở lại nhưng anh vẫn cố gắng để vượt qua nỗi đau anh mong thời gian sẽ quay trở lại như xưa để anh được ôm em giữ em trong lòng chỉ cần gặp em dù chỉ là trong phút giây vì anh yêu em anh sẽ mãi đợi chờ em bao nhiêu hy vọng dường như đã muốn khép lại hạnh phúc ngọt ngào giờ cũng muốn tan thành mảnh không biết giờ đây em đang ở nơi phương ơi còn nhớ và yêu anh nữa không hay em đã quên anh rồi anh đã từng khóc để cho vơi đi sầu đau anh đã từng khóc để quên hết những tháng ngày cô đơn mình anh vơi bao đêm dài mong nhớ nhưng anh vẫn cố gắng để vượt qua nỗi đau anh mong thời gian sẽ quay trở lại như xưa để anh được ôm em giữ em trong lòng chỉ cần gặp em dù chỉ là trong phút giây vì anh yêu em anh sẽ mãi đợi trước em chỉ cần gặp em dù chỉ là trong phút giây vì anh yêu em anh sẽ mãi đợi chờ em 

In [135]:
print("total songs: ", len(preprocessed_song))
print("mer mean: ", all['mer']/len(songs))
print("wer mean: ", all['wer']/len(songs))
print("wil mean: ", all['wil']/len(songs))
print("wip mean: ", all['wip']/len(songs))
print("cer mean: ", all['cer']/len(songs))

total songs:  982
mer mean:  0.41084223346226056
wer mean:  0.4439902621058862
wil mean:  0.5322655427197447
wip mean:  0.44973445728025524
cer mean:  0.34372632754541826


In [ ]:
1k
wer: 0.4439
cer 0.343

100
wer: 0.4922
cer: 0.3855

In [ ]:
100 samples

lyricwhiz:
- wer: 0.4922
- cer: 0.3855

whisper-small
- wer: 48.21
- cer: 31.61

whisper-base
- wer: 56.83
- cer: 37.31

whisper-medium
- wer: 36.82
- cer: 23.62

whisper-med-pp
- wer: 32.91
- cer: 20.86

whisper-med-demucs
- wer: 35.38
- cer: 22.8